In [143]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

# Import Data Set
bitcoin_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 0)
ethereum_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 1)
doge_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 2)
gold_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 3)
SP500_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 4)


In [144]:
# Linear Regression using SciKit Learn

# Import LinearRegression model from Scikit-Learn
from sklearn.linear_model import LinearRegression

# Create a Linear Regression object
linreg = LinearRegression()

In [77]:
#bitcoin_data.head()
#ethereum_data.head()
#doge_data.head()
#gold_data.info()
#SP500_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Asset   1260 non-null   object        
 1   Date    1260 non-null   datetime64[ns]
 2   Open    1260 non-null   object        
 3   High    1260 non-null   object        
 4   Low     1260 non-null   object        
 5   Close   1260 non-null   object        
 6   Volume  1260 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 69.0+ KB


In [146]:
# To remove one row where all values are '-'

gold_data = gold_data.drop(gold_data[gold_data.Date == "29/1/2018"].index)

# To remove 4 rows where corresponding cryptocurrencies do not have close value
gold_data = gold_data.drop(gold_data[gold_data.Date == "2020/10/13"].index)
gold_data = gold_data.drop(gold_data[gold_data.Date == "2020/10/12"].index)
gold_data = gold_data.drop(gold_data[gold_data.Date == "2020/10/9"].index)
gold_data = gold_data.drop(gold_data[gold_data.Date == "2020/4/17"].index)

SP500_data = SP500_data.drop(SP500_data[SP500_data.Date == "2020/10/13"].index)
SP500_data = SP500_data.drop(SP500_data[SP500_data.Date == "2020/10/12"].index)
SP500_data = SP500_data.drop(SP500_data[SP500_data.Date == "2020/10/9"].index)
SP500_data = SP500_data.drop(SP500_data[SP500_data.Date == "2020/4/17"].index)

#gold_data.info()

In [147]:
bitcoin_close = pd.DataFrame(bitcoin_data[["Date", "Close"]])
ethereum_close = pd.DataFrame(ethereum_data[["Date", "Close"]])
doge_close = pd.DataFrame(doge_data[["Date", "Close"]])
gold_data["Close"] = gold_data["Close"].astype('float')
gold_close = pd.DataFrame(gold_data[["Date", "Close"]])

SP500_close = pd.DataFrame(SP500_data[["Date", "Close"]])

In [148]:
# Merge BTC and ETH
close_data = bitcoin_close.merge(ethereum_close, left_on='Date', right_on='Date')
close_data = close_data.rename(columns={"Close_x" : "BTC_Close", "Close_y" : "ETH_Close"})


# Merge Doge with existing 2 asset classes
close_data = close_data.merge(doge_close, left_on='Date', right_on='Date')
close_data = close_data.rename(columns={"Close" : "Doge_Close"})

crypto_data = close_data

# Merge Gold with existing 3 asset classes
close_data = close_data.merge(gold_close, how='left', left_on='Date', right_on='Date')
close_data = close_data.rename(columns={"Close" : "Gold_Close"})

# Merge S&P500 with existing 4 asset classes
close_data = close_data.merge(SP500_close, how='left', left_on='Date', right_on='Date')
close_data = close_data.rename(columns={"Close" : "S&P500_Close"})

#close_data.head(20)
close_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1827 entries, 0 to 1826
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          1827 non-null   datetime64[ns]
 1   BTC_Close     1823 non-null   float64       
 2   ETH_Close     1823 non-null   float64       
 3   Doge_Close    1823 non-null   float64       
 4   Gold_Close    1257 non-null   float64       
 5   S&P500_Close  1255 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 99.9 KB


In [183]:
#crypto_data.head(200)

non_crypto_data = gold_close.merge(SP500_close, how = 'outer', left_on='Date', right_on='Date')
non_crypto_data = non_crypto_data.rename(columns={"Close_x" : "Gold_Close", "Close_y" : "S&P500_Close"})


# Merge Doge with existing 2 asset classes
#non_crypto_data = non_crypto_data.merge(doge_close, left_on='Date', right_on='Date')
#non_crypto_data = non_crypto_data.rename(columns={"Close" : "Doge_Close"})


#non_crypto_data.head(200)
non_crypto_data.info()

#non_c_log = np.log(non_crypto_data/non_crypto_data.shift())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1428 entries, 0 to 1427
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          1428 non-null   datetime64[ns]
 1   Gold_Close    1257 non-null   float64       
 2   S&P500_Close  1255 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 44.6 KB


In [186]:
is_na_noncrypto = non_crypto_data[non_crypto_data.isna().any(axis=1)]
#is_na = close_data[close_data.isna().any(axis=1)]

#is_na_noncrypto.info()
is_na_noncrypto.head(344)

,Date,Gold_Close,S&P500_Close
14,2021-11-25,1791.4,NaN
32,2021-10-31,1795.1,NaN
37,2021-10-24,1806.0,NaN
42,2021-10-17,1764.8,NaN
47,2021-10-10,1754.6,NaN
...,...,...,...
1423,2017-04-13,NaN,2328.95
1424,2017-04-07,NaN,2355.54
1425,2017-03-31,NaN,2362.72
1426,2017-03-24,NaN,2343.98


In [166]:
#non_crypto_data.info()
non_crypto_data = non_crypto_data.set_index('Date')
non_c_log = np.log(non_crypto_data/non_crypto_data.shift())

non_c_log

,Gold_Close,S&P500_Close
Date,,
2021-12-15,NaN,NaN
2021-12-14,0.004416,-0.016216
2021-12-13,0.008941,0.007499
2021-12-10,-0.001905,0.009178
2021-12-09,-0.004666,-0.009504
...,...,...
2016-12-21,0.002035,0.001865
2016-12-20,0.000354,0.002460
2016-12-19,0.007923,-0.003631


In [156]:
crypto_data = crypto_data.set_index('Date')
log_returns = np.log(crypto_data/crypto_data.shift())

log_returns

,BTC_Close,ETH_Close,Doge_Close
Date,,,
2021-12-15,NaN,NaN,NaN
2021-12-14,-0.047839,-0.070342,-0.005121
2021-12-13,0.002675,0.010302,-0.133991
2021-12-12,0.069441,0.088513,0.075319
2021-12-11,-0.014797,-0.012167,-0.005146
...,...,...,...
2016-12-19,-0.010244,-0.002120,-0.004577
2016-12-18,-0.002759,0.028566,0.000000
2016-12-17,0.000378,0.001432,0.004577


In [179]:
# Merge BTC and ETH
weekday_data = bitcoin_close.merge(ethereum_close, left_on='Date', right_on='Date')
weekday_data = weekday_data.rename(columns={"Close_x" : "BTC_Close", "Close_y" : "ETH_Close"})


# Merge Doge with existing 2 asset classes
weekday_data = weekday_data.merge(doge_close, left_on='Date', right_on='Date')
weekday_data = weekday_data.rename(columns={"Close" : "Doge_Close"})


# Merge Gold with existing 3 asset classes
weekday_data = weekday_data.merge(gold_close, left_on='Date', right_on='Date')
weekday_data = weekday_data.rename(columns={"Close" : "Gold_Close"})

# Merge S&P500 with existing 4 asset classes
weekday_data = weekday_data.merge(SP500_close, how = 'left', left_on='Date', right_on='Date')
weekday_data = weekday_data.rename(columns={"Close" : "S&P500_Close"})

#close_data.head(20)
weekday_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1257 entries, 0 to 1256
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          1257 non-null   datetime64[ns]
 1   BTC_Close     1257 non-null   float64       
 2   ETH_Close     1257 non-null   float64       
 3   Doge_Close    1257 non-null   float64       
 4   Gold_Close    1257 non-null   float64       
 5   S&P500_Close  1084 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 68.7 KB


In [181]:
is_na = weekday_data[weekday_data.isna().any(axis=1)]
#is_na = close_data[close_data.isna().any(axis=1)]

is_na.head(173)

,Date,BTC_Close,ETH_Close,Doge_Close,Gold_Close,S&P500_Close
14,2021-11-25,57274.679688,4274.743160,0.216326,1791.4,NaN
32,2021-10-31,61318.957031,4288.074220,0.280244,1795.1,NaN
37,2021-10-24,60930.835938,4087.903080,0.277048,1806.0,NaN
42,2021-10-17,61553.617188,3847.104490,0.237898,1764.8,NaN
47,2021-10-10,54771.578125,3425.852780,0.229805,1754.6,NaN
...,...,...,...,...,...,...
1178,2017-04-09,1187.869995,43.266998,0.000381,1251.1,NaN
1183,2017-04-02,1102.170044,48.748699,0.000489,1250.8,NaN
1188,2017-03-26,966.724976,50.521599,0.000272,1255.4,NaN
1193,2017-03-19,1036.739990,44.740101,0.000235,1233.6,NaN


In [118]:
#weekday_data.info()
#weekday_data.head(200)

In [119]:
#weekday_log_returns = np.log(weekday_data/weekday_data.shift())
#weekday_log_returns

In [121]:
#close_data

In [158]:
crypto_data.corr()

,BTC_Close,ETH_Close,Doge_Close
BTC_Close,1.000000,0.922727,0.785117
ETH_Close,0.922727,1.000000,0.875643
Doge_Close,0.785117,0.875643,1.000000


In [159]:
log_returns.corr()

,BTC_Close,ETH_Close,Doge_Close
BTC_Close,1.000000,0.689398,0.467352
ETH_Close,0.689398,1.000000,0.444276
Doge_Close,0.467352,0.444276,1.000000


In [170]:
close_data.corr()

,BTC_Close,ETH_Close,Doge_Close,Gold_Close,S&P500_Close
BTC_Close,1.000000,0.922727,0.785117,0.658878,0.907799
ETH_Close,0.922727,1.000000,0.875643,0.542455,0.857885
Doge_Close,0.785117,0.875643,1.000000,0.494999,0.765226
Gold_Close,0.658878,0.542455,0.494999,1.000000,0.813725
S&P500_Close,0.907799,0.857885,0.765226,0.813725,1.000000


In [167]:
#SP500_close.info()
#log_sp500 = np.log(SP500_close['Close']/SP500_close['Close'].shift)

#log_doge = np.log(doge_close['Close']/doge_close['Close'].shift)

non_c_log.corr()

,Gold_Close,S&P500_Close
Gold_Close,1.000000,-0.077692
S&P500_Close,-0.077692,1.000000


In [171]:
close_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1827 entries, 2021-12-15 to 2016-12-15
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BTC_Close     1823 non-null   float64
 1   ETH_Close     1823 non-null   float64
 2   Doge_Close    1823 non-null   float64
 3   Gold_Close    1257 non-null   float64
 4   S&P500_Close  1255 non-null   float64
dtypes: float64(5)
memory usage: 85.6 KB


In [176]:
weekday_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1084 entries, 2021-12-15 to 2016-12-15
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BTC_Close     1084 non-null   float64
 1   ETH_Close     1084 non-null   float64
 2   Doge_Close    1084 non-null   float64
 3   Gold_Close    1084 non-null   float64
 4   S&P500_Close  1084 non-null   float64
dtypes: float64(5)
memory usage: 50.8 KB


In [182]:
#non_crypto_data.info()
weekday_data = weekday_data.set_index('Date')
all_close_log = np.log(weekday_data/weekday_data.shift())

#all_close_log

In [175]:
close_data.corr()

,BTC_Close,ETH_Close,Doge_Close,Gold_Close,S&P500_Close
BTC_Close,1.000000,0.922727,0.785117,0.658878,0.907799
ETH_Close,0.922727,1.000000,0.875643,0.542455,0.857885
Doge_Close,0.785117,0.875643,1.000000,0.494999,0.765226
Gold_Close,0.658878,0.542455,0.494999,1.000000,0.813725
S&P500_Close,0.907799,0.857885,0.765226,0.813725,1.000000


In [173]:
all_close_log.corr()

,BTC_Close,ETH_Close,Doge_Close,Gold_Close,S&P500_Close
BTC_Close,1.000000,0.662547,0.478095,0.077648,0.178809
ETH_Close,0.662547,1.000000,0.457201,0.078752,0.202566
Doge_Close,0.478095,0.457201,1.000000,0.046836,0.089072
Gold_Close,0.077648,0.078752,0.046836,1.000000,-0.077692
S&P500_Close,0.178809,0.202566,0.089072,-0.077692,1.000000


In [174]:
log_returns.corr()

,BTC_Close,ETH_Close,Doge_Close
BTC_Close,1.000000,0.689398,0.467352
ETH_Close,0.689398,1.000000,0.444276
Doge_Close,0.467352,0.444276,1.000000


## Set 1 : Crypto + Gold

In [187]:
# Merge Gold with existing 3 crypto assets
crypto_gold = crypto_data.merge(gold_close, how='left', left_on='Date', right_on='Date')
crypto_gold = crypto_gold.rename(columns={"Close" : "Gold_Close"})


#close_data.head(20)
crypto_gold.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1827 entries, 0 to 1826
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1827 non-null   datetime64[ns]
 1   BTC_Close   1823 non-null   float64       
 2   ETH_Close   1823 non-null   float64       
 3   Doge_Close  1823 non-null   float64       
 4   Gold_Close  1257 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 85.6 KB


In [189]:
crypto_gold = crypto_gold.set_index('Date')
log_crypto_gold = np.log(crypto_gold/crypto_gold.shift())

log_crypto_gold

,BTC_Close,ETH_Close,Doge_Close,Gold_Close
Date,,,,
2021-12-15,NaN,NaN,NaN,NaN
2021-12-14,-0.047839,-0.070342,-0.005121,0.004416
2021-12-13,0.002675,0.010302,-0.133991,0.008941
2021-12-12,0.069441,0.088513,0.075319,NaN
2021-12-11,-0.014797,-0.012167,-0.005146,NaN
...,...,...,...,...
2016-12-19,-0.010244,-0.002120,-0.004577,0.007923
2016-12-18,-0.002759,0.028566,0.000000,NaN
2016-12-17,0.000378,0.001432,0.004577,NaN


In [191]:
log_crypto_gold.corr()
#crypto_gold.corr()

,BTC_Close,ETH_Close,Doge_Close,Gold_Close
BTC_Close,1.000000,0.689398,0.467352,0.076692
ETH_Close,0.689398,1.000000,0.444276,0.072958
Doge_Close,0.467352,0.444276,1.000000,0.027722
Gold_Close,0.076692,0.072958,0.027722,1.000000


## Set 2 : Crypto + S&P500

In [188]:
# Merge S&P500 with existing 3 crypto assets
crypto_sp500 = crypto_data.merge(SP500_close, how='left', left_on='Date', right_on='Date')
crypto_sp500 = crypto_sp500.rename(columns={"Close" : "S&P500_Close"})

#close_data.head(20)
crypto_sp500.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1827 entries, 0 to 1826
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          1827 non-null   datetime64[ns]
 1   BTC_Close     1823 non-null   float64       
 2   ETH_Close     1823 non-null   float64       
 3   Doge_Close    1823 non-null   float64       
 4   S&P500_Close  1255 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 85.6 KB


In [190]:
crypto_sp500 = crypto_sp500.set_index('Date')
log_crypto_sp500 = np.log(crypto_sp500/crypto_sp500.shift())

log_crypto_sp500

,BTC_Close,ETH_Close,Doge_Close,S&P500_Close
Date,,,,
2021-12-15,NaN,NaN,NaN,NaN
2021-12-14,-0.047839,-0.070342,-0.005121,-0.016216
2021-12-13,0.002675,0.010302,-0.133991,0.007499
2021-12-12,0.069441,0.088513,0.075319,NaN
2021-12-11,-0.014797,-0.012167,-0.005146,NaN
...,...,...,...,...
2016-12-19,-0.010244,-0.002120,-0.004577,-0.003631
2016-12-18,-0.002759,0.028566,0.000000,NaN
2016-12-17,0.000378,0.001432,0.004577,NaN


In [192]:
log_crypto_sp500.corr()
#crypto_sp500.corr()

,BTC_Close,ETH_Close,Doge_Close,S&P500_Close
BTC_Close,1.000000,0.689398,0.467352,0.220237
ETH_Close,0.689398,1.000000,0.444276,0.231029
Doge_Close,0.467352,0.444276,1.000000,0.110418
S&P500_Close,0.220237,0.231029,0.110418,1.000000
